In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# This code snippet does not run on its own.  This is intended to be copy / pasted into Jupyter Notebook.
from config import gkey



In [2]:
fire_df = pd.read_csv ('Building_Fire.csv')
#branch
fire_df 

,ID,INCIDENT,DATE,TIME,ADDRESS,ZIP,BATTALION/SHIFT,MAPSCO,COUNCIL_DISTRICT,CENSUS_TRACT,STRUCTURE_TYPE,NO_OF_STORIES,CONSTRUCTION,CAUSE,TOTAL_VALUE,STRUCTURE_LOSS,CONTENTS_LOSS,TOTAL_SAVED,SPRINKER_SYS,ALARM_SYS
0,17961.0,2.019261e+09,10/29/2019,08:21 PM,1325 S MARSALIS AVE,75216.0,6/A,55J,4.0,5400.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,NaN,NaN
1,17962.0,2.019261e+09,10/30/2019,03:42 AM,10728 LATHROP DR,75229.0,7/A,23H,13.0,9702.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,NaN,NaN
2,17963.0,2.019261e+09,10/30/2019,10:47 AM,6205 S R L THORNTON SERV,75241.0,5/B,64R,3.0,11200.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,NaN,NaN
3,17964.0,2.019261e+09,10/30/2019,05:32 PM,12044 WILLOWDELL DR,75243.0,2/B,16X,10.0,7804.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,NaN,NaN
4,17965.0,2.019261e+09,10/30/2019,11:03 AM,2212 LAPSLEY ST,75212.0,9/B,42U,6.0,10602.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,18560.0,2.020237e+09,10/20/2019,06:11 PM,1209 ROUND TABLE DR,75247.0,9/A,33N,6.0,10000.0,PALLET OF STONE,NaN,NaN,INCENDIARY,838940.0,0.0,-10000.0,828940.0,NaN,NO
1030,NaN,NaN,02/06/2020,03:27 PM,2325 HILLGLENN RD,75228.0,4/B-E,38R-E,NaN,NaN,FENCE,NaN,WOOD,EXPOSURE (UNDETERMINED),10000.0,-500.0,0.0,9500.0,NaN,UNK
1031,NaN,2.021025e+09,02/01/2020,04:29 PM,1542 FORDHAM RD,75216.0,5/C,65B,4.0,5700.0,DWELLING,1.0,WOOD,UNDETERMINED,171340.0,-2000.0,-40.0,169300.0,NO,NO
1032,NaN,2.021025e+09,02/01/2020,02:32 PM,5500 JULIUS SCHEPPS FWY NB,75216.0,1/C,56B,7.0,8603.0,VEHICLE,2011.0,NISSAN ALTIMA,INCENDIARY (ATTEMPTED SUICIDE),5200.0,-4000.0,0.0,1200.0,NaN,NO


In [3]:
df = fire_df.drop("MAPSCO", axis='columns')
new_fire_df = df.drop("CENSUS_TRACT", axis='columns')
new_fire_df

,ID,INCIDENT,DATE,TIME,ADDRESS,ZIP,BATTALION/SHIFT,COUNCIL_DISTRICT,STRUCTURE_TYPE,NO_OF_STORIES,CONSTRUCTION,CAUSE,TOTAL_VALUE,STRUCTURE_LOSS,CONTENTS_LOSS,TOTAL_SAVED,SPRINKER_SYS,ALARM_SYS
0,17961.0,2.019261e+09,10/29/2019,08:21 PM,1325 S MARSALIS AVE,75216.0,6/A,4.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,NaN,NaN
1,17962.0,2.019261e+09,10/30/2019,03:42 AM,10728 LATHROP DR,75229.0,7/A,13.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,NaN,NaN
2,17963.0,2.019261e+09,10/30/2019,10:47 AM,6205 S R L THORNTON SERV,75241.0,5/B,3.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,NaN,NaN
3,17964.0,2.019261e+09,10/30/2019,05:32 PM,12044 WILLOWDELL DR,75243.0,2/B,10.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,NaN,NaN
4,17965.0,2.019261e+09,10/30/2019,11:03 AM,2212 LAPSLEY ST,75212.0,9/B,6.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,18560.0,2.020237e+09,10/20/2019,06:11 PM,1209 ROUND TABLE DR,75247.0,9/A,6.0,PALLET OF STONE,NaN,NaN,INCENDIARY,838940.0,0.0,-10000.0,828940.0,NaN,NO
1030,NaN,NaN,02/06/2020,03:27 PM,2325 HILLGLENN RD,75228.0,4/B-E,NaN,FENCE,NaN,WOOD,EXPOSURE (UNDETERMINED),10000.0,-500.0,0.0,9500.0,NaN,UNK
1031,NaN,2.021025e+09,02/01/2020,04:29 PM,1542 FORDHAM RD,75216.0,5/C,4.0,DWELLING,1.0,WOOD,UNDETERMINED,171340.0,-2000.0,-40.0,169300.0,NO,NO
1032,NaN,2.021025e+09,02/01/2020,02:32 PM,5500 JULIUS SCHEPPS FWY NB,75216.0,1/C,7.0,VEHICLE,2011.0,NISSAN ALTIMA,INCENDIARY (ATTEMPTED SUICIDE),5200.0,-4000.0,0.0,1200.0,NaN,NO


In [4]:
'''
drop columns:
MAPSCO
Census Tract
drop rows:
stories < 100
total_saved > 0
total_value > 1


There are 621 rows

'''

clean_fire_df = new_fire_df.loc[(new_fire_df["NO_OF_STORIES"] < 100) & (new_fire_df["TOTAL_SAVED"]> 0) \
                                & (new_fire_df["TOTAL_VALUE"] > 1 )]
clean_fire_df

,ID,INCIDENT,DATE,TIME,ADDRESS,ZIP,BATTALION/SHIFT,COUNCIL_DISTRICT,STRUCTURE_TYPE,NO_OF_STORIES,CONSTRUCTION,CAUSE,TOTAL_VALUE,STRUCTURE_LOSS,CONTENTS_LOSS,TOTAL_SAVED,SPRINKER_SYS,ALARM_SYS
0,17961.0,2.019261e+09,10/29/2019,08:21 PM,1325 S MARSALIS AVE,75216.0,6/A,4.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,NaN,NaN
1,17962.0,2.019261e+09,10/30/2019,03:42 AM,10728 LATHROP DR,75229.0,7/A,13.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,NaN,NaN
2,17963.0,2.019261e+09,10/30/2019,10:47 AM,6205 S R L THORNTON SERV,75241.0,5/B,3.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,NaN,NaN
3,17964.0,2.019261e+09,10/30/2019,05:32 PM,12044 WILLOWDELL DR,75243.0,2/B,10.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,NaN,NaN
4,17965.0,2.019261e+09,10/30/2019,11:03 AM,2212 LAPSLEY ST,75212.0,9/B,6.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,18414.0,2.020199e+09,09/03/2020,02:12 AM,5024 URBAN AVE,75227.0,8/A,7.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,261000.0,-15000.0,-2500.0,243500.0,NO,YES
1021,18459.0,2.020212e+09,09/19/2020,08:49 AM,5307 GASTON AVE,75214.0,3/C,14.0,APARTMENTS,2.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,2240263.0,-2500.0,0.0,2237763.0,NaN,UNK
1025,18496.0,2.020216e+09,09/24/2020,07:11 PM,218 W FERRIS BRANCH BLVD,75243.0,4/B,10.0,APARTMENTS,3.0,WOOD,EQUIPMENT UNATTENDED,7357830.0,-200.0,0.0,7357630.0,YES,UKN
1028,18514.0,2.020220e+09,09/30/2020,07:15 AM,14770 LASATER RD,75253.0,8/B,8.0,SHED,1.0,METAL,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,1500.0,-500.0,-100.0,900.0,NO,NO


In [10]:
#clean_fire_df.loc[(clean_fire_df["TOTAL_SAVED"] > 0)]

total_save = clean_fire_df["TOTAL_SAVED"]
total_value = clean_fire_df["TOTAL_VALUE"]

percentage_saved = total_save / total_value * 100

percentage_loss = (100 - percentage_saved).round(2)


percentage_loss

clean_fire_df["PERCENTAGE LOSS"] = percentage_loss

#clean_fire_df=clean_fire_df.set_index('ID')
#clean_fire_df=clean_fire_df.reset_index()
clean_fire_df

,ID,INCIDENT,DATE,TIME,ADDRESS,ZIP,BATTALION/SHIFT,COUNCIL_DISTRICT,STRUCTURE_TYPE,NO_OF_STORIES,CONSTRUCTION,CAUSE,TOTAL_VALUE,STRUCTURE_LOSS,CONTENTS_LOSS,TOTAL_SAVED,SPRINKER_SYS,ALARM_SYS,PERCENTAGE LOSS
0,17961.0,2.019261e+09,10/29/2019,08:21 PM,1325 S MARSALIS AVE,75216.0,6/A,4.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,NaN,NaN,3.01
1,17962.0,2.019261e+09,10/30/2019,03:42 AM,10728 LATHROP DR,75229.0,7/A,13.0,DWELLING,1.0,WOOD,STORM,535470.0,-10000.0,0.0,525470.0,NaN,NaN,1.87
2,17963.0,2.019261e+09,10/30/2019,10:47 AM,6205 S R L THORNTON SERV,75241.0,5/B,3.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,NaN,NaN,6.54
3,17964.0,2.019261e+09,10/30/2019,05:32 PM,12044 WILLOWDELL DR,75243.0,2/B,10.0,DWELLING,1.0,WOOD,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,NaN,NaN,9.47
4,17965.0,2.019261e+09,10/30/2019,11:03 AM,2212 LAPSLEY ST,75212.0,9/B,6.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,NaN,NaN,65.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,18414.0,2.020199e+09,09/03/2020,02:12 AM,5024 URBAN AVE,75227.0,8/A,7.0,DWELLING,1.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,261000.0,-15000.0,-2500.0,243500.0,NO,YES,6.70
617,18459.0,2.020212e+09,09/19/2020,08:49 AM,5307 GASTON AVE,75214.0,3/C,14.0,APARTMENTS,2.0,WOOD,UNSPECIFIED SHORT CIRCUIT ARC,2240263.0,-2500.0,0.0,2237763.0,NaN,UNK,0.11
618,18496.0,2.020216e+09,09/24/2020,07:11 PM,218 W FERRIS BRANCH BLVD,75243.0,4/B,10.0,APARTMENTS,3.0,WOOD,EQUIPMENT UNATTENDED,7357830.0,-200.0,0.0,7357630.0,YES,UKN,0.00
619,18514.0,2.020220e+09,09/30/2020,07:15 AM,14770 LASATER RD,75253.0,8/B,8.0,SHED,1.0,METAL,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,1500.0,-500.0,-100.0,900.0,NO,NO,40.00


In [11]:
lats = []
lngs = []
# geocoordinates
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

for index, row in clean_fire_df.iterrows():
    address = row['ADDRESS'] + ", Dallas, TX"
    
    # set up a parameters dictionary
    params = {
        "address": address.replace(" ", "%20"),
        "key": gkey
    }
    
    geo_data = requests.get(base_url, params=params).json()
    #geo_data = response.json()
    # Extract latitude and longitude
    #lat = geo_data["results"][0]["geometry"]["location"]["lat"]
    #lng = geo_data["results"][0]["geometry"]["location"]["lng"]
    
    clean_fire_df.loc[index, "Lat"] = geo_data["results"][0]["geometry"]["location"]["lat"]
    clean_fire_df.loc[index, "Lng"] = geo_data["results"][0]["geometry"]["location"]["lng"]


    #lats.append(lat)
    #lngs.append(lng)

        
    
#clean_fire_df['Lat'] = lats
#clean_fire_df['Lng'] = lngs


IndexError: list index out of range

1

In [15]:
clean_fire_df.head()

,ID,INCIDENT,DATE,TIME,ADDRESS,ZIP,BATTALION/SHIFT,COUNCIL_DISTRICT,STRUCTURE_TYPE,NO_OF_STORIES,...,CAUSE,TOTAL_VALUE,STRUCTURE_LOSS,CONTENTS_LOSS,TOTAL_SAVED,SPRINKER_SYS,ALARM_SYS,PERCENTAGE LOSS,Lat,Lng
0,17961.0,2.019261e+09,10/29/2019,08:21 PM,1325 S MARSALIS AVE,75216.0,6/A,4.0,DWELLING,1.0,...,UNSPECIFIED SHORT CIRCUIT ARC,166370.0,-5000.0,0.0,161370.0,NaN,NaN,3.01,32.776664,-96.796988
1,17962.0,2.019261e+09,10/30/2019,03:42 AM,10728 LATHROP DR,75229.0,7/A,13.0,DWELLING,1.0,...,STORM,535470.0,-10000.0,0.0,525470.0,NaN,NaN,1.87,NaN,NaN
2,17963.0,2.019261e+09,10/30/2019,10:47 AM,6205 S R L THORNTON SERV,75241.0,5/B,3.0,DWELLING,1.0,...,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,306000.0,-5000.0,-15000.0,286000.0,NaN,NaN,6.54,NaN,NaN
3,17964.0,2.019261e+09,10/30/2019,05:32 PM,12044 WILLOWDELL DR,75243.0,2/B,10.0,DWELLING,1.0,...,HEAT SOURCE TOO CLOSE TO COMBUSTIBLES,126710.0,-8000.0,-4000.0,114710.0,NaN,NaN,9.47,NaN,NaN
4,17965.0,2.019261e+09,10/30/2019,11:03 AM,2212 LAPSLEY ST,75212.0,9/B,6.0,DWELLING,1.0,...,UNSPECIFIED SHORT CIRCUIT ARC,84000.0,-45000.0,-10000.0,29000.0,NaN,NaN,65.48,NaN,NaN


In [ ]:
address

In [ ]:
address.replace(" ", "%20")